The notebook is meant to help the user experiment with different models and features. This notebook assumes that there is a saved csv called 'filteredAggregateData.csv' somewhere on your local harddrive. The location must be specified below.

The cell imports all of the relevant packages.

In [145]:
############## imports
# general
import statistics
import datetime
from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
# saving models
# from sklearn.externals import joblib
import joblib


Imports the API. 'APILoc' is the location of 'API.py' on your local harddrive.

In [146]:
# import the API
APILoc = r"C:\Users\thejo\Documents\school\AI in AG research\API"

import sys
sys.path.insert(0, APILoc)

from API import *

Load the dataset. Note that the location of the dataset must be specified.

In [147]:
# get aggregate data
aggDataLoc = r'C:\Users\thejo\Documents\school\AI in AG research\experiment\aggregateData_MS_KY_GA.csv'
#aggDataLoc = r'C:\Users\thejo\Documents\school\AI in AG research\experiment\aggregateDataWithVariety.csv'
#targetDataLoc = r'C:\Users\thejo\Documents\school\AI in AG research\experiment\aggregateData_GAonly_Annual_final.csv'

aggDf = pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
#targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)


Test to see if the dataset was loaded properly. A table of the first 5 datapoints should appear.

In [18]:
aggDf.head()
#targetDf.head()

,Unnamed: 0,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Harvested in Sown Year,First Date of Season,Percent Cover (%)
0,0,Georgia,Athens,10/10/2007,TS 4010,5/6/2008,127,2.02,209,209,922.90,98.288,14.886400,8.691000,21.120200,0.15088,0,1,NaN
1,1,Georgia,Athens,10/10/2007,TS 4010,6/20/2008,172,0.47,254,45,1045.84,85.106,23.029778,16.898889,29.451333,0.12520,0,0,NaN
2,2,Georgia,Athens,10/10/2007,TS 4010,8/28/2008,241,0.10,323,69,953.74,134.634,25.709800,20.641600,31.935200,0.10770,0,0,NaN
3,3,Georgia,Athens,10/10/2007,TS 4010,12/5/2008,340,0.76,422,99,622.84,95.254,9.790200,3.943600,16.041000,0.13408,0,0,90.915344
4,4,Georgia,Athens,10/10/2007,BaraWet 501,5/6/2008,127,1.66,209,209,922.90,98.288,14.886400,8.691000,21.120200,0.15088,0,1,NaN


Filter out features that will not be made available for feature selection. All of the features in the list 'XColumnsToKeep' will be made available for feature selection. The features to include are: <br>
"Julian Day" <br>
"Time Since Sown (Days)" <br>
"Time Since Last Harvest (Days)" <br>
"Total Radiation (MJ/m^2)" <br>
"Total Rainfall (mm)" <br>
"Avg Air Temp (C)" <br>
"Avg Min Temp (C)" <br>
"Avg Max Temp (C)"<br>
"Avg Soil Moisture (%)"<br>
"Day Length (hrs)"<br>
"Percent Cover (%)"<br>

In [148]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Yield (tons/acre)"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
#yDf = targetDf[yColumnsToKeep]
yDf = aggDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

Test to see if the features dataframe and the target dataframe were successfully made.

In [10]:
xDf.head()

,Julian Day,Time Since Sown (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%)
0,127,209,922.90,98.288,14.886400,8.691000,21.120200,0.15088
1,172,254,1045.84,85.106,23.029778,16.898889,29.451333,0.12520
2,241,323,953.74,134.634,25.709800,20.641600,31.935200,0.10770
3,340,422,622.84,95.254,9.790200,3.943600,16.041000,0.13408
4,127,209,922.90,98.288,14.886400,8.691000,21.120200,0.15088


In [127]:
yDf.head()

,Yield (tons/acre)
0,3.35
1,2.99
2,3.08
3,3.10
4,3.13


Lets now define the parameters that will be used to run the machine learning experiments. Note that parameter grids could be made that will allow sci-kit learn to use a 5-fold gridsearch to find the model's best hyperparameters. The parameter grids that are defined here will specify the possible values for the grid search. <br>
<br>
Once the parameter grids are defined, a list of tuples must also be defined. The tuples must take the form of: <br>
(sci-kit learn model, appropriate parameter grid, name of the file to be saved). <br>
<br>
Then the number of iterations should be made. This is represented by the variable 'N'. Each model will be evaluated N times (via N-fold cross validation), and the average results of the models over those N iterations will be returned. <br>
<br>
'workingDir' is the directory in which all of the results will be saved. <br>
<br>
'numFeatures' is the number of features that will be selected (via feature selection).

In [155]:

# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["mae"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['mae'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [(RandomForestRegressor(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsRegressor(), knnParamGrid, "knnTup.pkl"),
              (SVR(), svrParamGrid, "svrTup.pkl"),
             #(MLPRegressor(), nnParamGrid, "nnTup.pkl")]#,
             (LinearRegression(), linRegParamGrid, "linRegTup.pkl"),
             (BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             (DecisionTreeRegressor(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = r"C:\Users\thejo\Documents\school\AI in AG research\experiment"
numFeatures = 8 # 11


This cell will run the tests and save the results.

In [156]:
saveMLResults(N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  rfTup
Avg MAE:  546289.95
Avg R squared:  0.082
Best MAE:  1668.613
Best R squared:  0.644
Parameters of the best model:  {'model__criterion': 'mae', 'model__max_depth': 20, 'model__n_estimators': 10}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
model:  knnTup
Avg MAE:  546595.773
Avg R squared:  -1.1
Best MAE:  2371.145
Best R squared:  0.606
Parameters of the best model:  {'model__leaf_size': 5, 'model__n_neighbors': 2, 'model__weights': 'uniform'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
model:  svrTup
Avg MAE:  545885.93
Avg R squared:  -0.197
Best MAE:  1557.557
Best R squared:  0.863
Parameters of the best model:  {'model__C': 10.0, 'mode